In [14]:
import cv2
import numpy as np
import requests
from urllib.parse import urlencode
from pathlib import Path

pwd = Path.cwd()
dataset_path = pwd / "dataset.mp4"


In [15]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
video_on_yandex_disk = 'https://disk.yandex.ru/i/MFfiANmh1QulrQ'
final_url = base_url + urlencode(dict(public_key=video_on_yandex_disk))
response = requests.get(final_url)
download_url = response.json()['href']


In [16]:
ftd = dict(  # files to download
    names=["yolov3-spp.weights", "coco.names", "yolov3-spp.cfg", "dataset.mp4"],
    links=[
        "https://pjreddie.com/media/files/yolov3-spp.weights",
        "https://github.com/pjreddie/darknet/raw/master/data/coco.names",
        "https://github.com/pjreddie/darknet/raw/master/cfg/yolov3-spp.cfg",
        download_url,
    ],
)


def download(name: str, link: str) -> None:
    print(f"Downloading {name}, wait...")
    r = requests.get(link, allow_redirects=True)
    with open(name, "wb") as f:
        f.write(r.content)
    print(f"Done ({name}).")


for name, link in zip(ftd["names"], ftd["links"]):
    if not (pwd / name).exists():  # Проверка на наличие нужных файлов
        download(name, link)


Done (dataset.mp4).


In [17]:
cap = cv2.VideoCapture(str(dataset_path))  # для вывода видео
if not cap.isOpened():
    raise Exception("Can't open video")

whT = 320  # idk what is that
confThreshold = 0.5
nmsThreshold = 0.3


In [18]:
classesFile = "coco.names"  # для классов YOLO штука (чтобы искать не только котов, а и машин, например)
classNames = []
with open(classesFile, "rt") as f:
    classNames = f.read().rstrip("\n").split("\n")

modelConfiguration = "yolov3-spp.cfg"  # подгрузка YOLO3-spp
modelWeights = "yolov3-spp.weights"
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)


In [19]:
# функция, для нахождения котиков UwU :з
def findObjects(outputs, img):
    hT, wT, cT = img.shape
    bbox = []
    classIds = []
    confs = []

    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w, h = int(det[2] * wT), int(det[3] * hT)
                x, y = int(det[0] * wT) - w / 2, int(det[1] * hT) - h / 2
                bbox.append([x, y, w, h])
                classIds.append(classId)
                confs.append(float(confidence))
    indices = cv2.dnn.NMSBoxes(bbox, confs, confThreshold, nmsThreshold)
    for i in indices:
        if classNames[classIds[i]] == "cat":
            box = bbox[i]
            x, y, w, h = box[0], box[1], box[2], box[3]
            cv2.rectangle(
                img, (int(x), int(y)), (int(x + w), int(y + h)), (255, 0, 0), 2
            )
            cv2.putText(
                img,
                f"Cat {int(confs[i]*100)}%",
                (int(x), int(y - 10)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (255, 0, 255),
                2,
            )


In [20]:
for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))-1):
    _, img = cap.read()
    blob = cv2.dnn.blobFromImage(img, 1 / 255, (whT, whT), [0, 0, 0], 1, crop=False)
    net.setInput(blob)
    layerNames = net.getLayerNames()
    outputNames = [layerNames[i - 1] for i in net.getUnconnectedOutLayers()]
    outputs = net.forward(outputNames)

    findObjects(outputs, img)

    cv2.imshow("Image", img)
    cv2.waitKey(1)

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 